<a href="https://colab.research.google.com/github/financieras/math_for_ai/blob/main/228_ejemplo_del_taxista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fórmula del perceptrón

En el perceptrón, la operación principal consiste en multiplicar los pesos $W$ por las entradas $X$ y sumar los resultados, lo que se conoce como producto escalar o suma ponderada. Esto determina el grado de activación de la neurona artificial.

## Motivo para añadir el 1 a $X$

Se añade un "1" extra al vector de entrada $X$ para incorporar el sesgo o bias en la fórmula del perceptrón. El sesgo permite desplazar el umbral de activación y, matemáticamente, se representa como un peso adicional multiplicado por ese "1". Esto simplifica la notación y permite expresar toda la operación como un único producto matricial.

## Dimensiones de $W$ y $X$

- Si $X$ es un vector columna con $m$ características, entonces, al añadir el sesgo, $X$ tendrá dimensión $(m+1) \times 1$. Es decir, es una columna con todas las características más el valor 1 añadido al inicio.
- El vector de pesos $W$ debe tener también dimensión $(m+1) \times 1$: un peso para cada característica más uno para el sesgo.

## ¿Es necesario usar la transpuesta?

La transpuesta es necesaria en la notación matricial para asegurar que las dimensiones sean compatibles en el producto. Por ejemplo, la operación se expresa como:

$$y = W^T X$$

Donde $W^T$ (transpuesta de $W$) es un vector fila de dimensión $1 \times (m+1)$ y $X$ es un vector columna de dimensión $(m+1) \times 1$, lo que resulta en un escalar.

## Resumen de la operación

- Se multiplica cada entrada por su peso correspondiente y se suma el resultado.
- El sesgo se añade multiplicando un peso extra por 1, dando flexibilidad para ajustar el umbral.
- Tanto $W$ como $X$ deben tener dimensiones compatibles tras añadir el sesgo.
- La notación con transpuesta es estándar en machine learning para operaciones matriciales.

Esto convierte la fórmula completa en:

$$y = W^T X$$

donde:
- $X = [1, x_1, x_2, \ldots, x_m]^T$
- $W = [w_0, w_1, w_2, \ldots, w_m]^T$

donde $w_0$ es el sesgo (bias).

Así, la suma de los pesos de entrada y el sesgo se combinan en un solo producto escalar.

---

## Ejemplo del taxista: Una carrera (una muestra)

[Hoja de Cálculo](https://docs.google.com/spreadsheets/d/1RHwkhmq2qKMV5LIjMt43g0VHawsT6M0-rjk0UhjqsC4/edit?usp=sharing)

Vamos a ilustrar cómo funciona un perceptrón usando el ejemplo de calcular el importe de una carrera de taxi.

### Tarifa del taxi

La tarifa tiene dos componentes:
- **Bajada de bandera:** 2,50 €
- **Precio por kilómetro:** 1,30 €

La ecuación para calcular el importe es:

$$y = 2.5 + 1.3 \cdot x$$

donde $x$ son los kilómetros recorridos e $y$ es el importe de la carrera.

### Representación matricial para una carrera

Supongamos una carrera de 10 km:

In [ ]:
import numpy as np

# Una carrera de 10 km
x = np.array([10])

# Añadimos el 1 para la bajada de bandera
x_ampliado = np.array([1, 10])  # shape: (2,)

# Pesos: [bajada_bandera, precio_por_km]
W = np.array([2.5, 1.3])  # shape: (2,)

# Calculamos el importe
importe = W @ x_ampliado

print("Importe de la carrera:", importe)  # 15.5 €

Importe de la carrera: 15.5


En notación matricial:

$$y = W^T X = \begin{bmatrix} 2.5 & 1.3 \end{bmatrix} \begin{bmatrix} 1 \\ 10 \end{bmatrix} = 15.5$$

**Dimensiones:** $W^T_{1 \times 2} \cdot X_{2 \times 1} = y_{1 \times 1}$ (un escalar)

Este es el caso fundamental: **un perceptrón procesando una entrada para producir una salida**.

---

## Múltiples carreras: Procesamiento en batch

En machine learning, rara vez procesamos una sola muestra a la vez. En la práctica, procesamos **mini-batches** (lotes) de múltiples muestras simultáneamente. Esto es mucho más eficiente computacionalmente y es fundamental para el entrenamiento de redes neuronales.

Veamos cómo se vería con 4 carreras de taxi:

In [ ]:
import numpy as np

# Kilómetros recorridos en 4 carreras diferentes
X = np.array([10, 4, 25, 20])

# Ampliamos X añadiendo una fila de unos (para la bajada de bandera)
X_ampliado = np.vstack([np.ones(X.shape), X])

# Pesos: [bajada_bandera, precio_por_km]
W = np.array([2.5, 1.3])

# Calculamos los importes de las 4 carreras simultáneamente
importes = W @ X_ampliado

print("X_ampliado:")
print(X_ampliado)
print("\nImportes de cada carrera:")
print(importes)
print("\nTotal facturado:")
print(np.sum(importes))

X_ampliado:
[[ 1.  1.  1.  1.]
 [10.  4. 25. 20.]]

Importes de cada carrera:
[15.5  7.7 35.  28.5]

Total facturado:
86.7


En notación matricial:

$$Y = W^T X = \begin{bmatrix} 2.5 & 1.3 \end{bmatrix} \begin{bmatrix} 1 & 1 & 1 & 1 \\ 10 & 4 & 25 & 20 \end{bmatrix} = \begin{bmatrix} 15.5 & 7.7 & 35 & 28.5 \end{bmatrix}$$

**Dimensiones:** $W^T_{1 \times 2} \cdot X_{2 \times 4} = Y_{1 \times 4}$

Ahora obtenemos 4 salidas (una por cada carrera), calculadas todas al mismo tiempo.

### Nota sobre NumPy y la transpuesta

NumPy es flexible con vectores 1D. Por eso, tanto `W @ X_ampliado` como `W.T @ X_ampliado` funcionan y dan el mismo resultado:

In [ ]:
print("Importes:", W @ X_ampliado)
print("Importes:", W.T @ X_ampliado)

Importes: [15.5  7.7 35.  28.5]
Importes: [15.5  7.7 35.  28.5]


Sin embargo, la notación formal en machine learning usa la transpuesta $W^T$ para mantener la consistencia matemática, especialmente cuando trabajamos con matrices multidimensionales.

---

## Añadiendo más características: Múltiples variables

Ahora consideremos que la tarifa del taxi también incluye un cargo de 0,50 € por cada minuto de espera.

La nueva ecuación es:

$$y = 2.5 + 1.3 x_1 + 0.5 x_2$$

donde:
- $x_1$ son los kilómetros recorridos
- $x_2$ son los minutos de espera
- $y$ es el importe de la carrera

Esta ecuación sigue siendo lineal, pero ahora representa un plano en 3D.

### Una carrera con dos características

In [ ]:
import numpy as np

# Una carrera: 10 km y 15 minutos de espera
x = np.array([10, 15])

# Añadimos el 1 para el sesgo
x_ampliado = np.array([1, 10, 15])

# Pesos: [bajada_bandera, precio_km, precio_minuto]
W = np.array([2.5, 1.3, 0.5])

# Calculamos el importe
importe = W @ x_ampliado

print("Importe de la carrera:", importe)  # 23.0 €

Importe de la carrera: 23.0


### Batch de 4 carreras con dos características

In [ ]:
# Matriz con km y minutos de 4 carreras diferentes
X = np.array([[10, 4, 25, 20],   # kilómetros
              [15, 0,  5, 10]])   # minutos de espera

# Ampliamos X añadiendo una fila de unos
X_ampliado = np.vstack([np.ones(X.shape[1]), X])

# Pesos: [bajada_bandera, precio_km, precio_minuto]
W = np.array([2.5, 1.3, 0.5])

# Calculamos los importes de las 4 carreras
importes = W @ X_ampliado

print("X_ampliado:")
print(X_ampliado)
print("\nImportes de cada carrera:")
print(importes)
print("\nTotal facturado:")
print(np.sum(importes))

X_ampliado:
[[ 1.  1.  1.  1.]
 [10.  4. 25. 20.]
 [15.  0.  5. 10.]]

Importes de cada carrera:
[23.   7.7 37.5 33.5]

Total facturado:
101.7


En notación matricial:

$$Y = W^T X = \begin{bmatrix} 2.5 & 1.3 & 0.5 \end{bmatrix} \begin{bmatrix} 1 & 1 & 1 & 1 \\ 10 & 4 & 25 & 20 \\ 15 & 0 & 5 & 10 \end{bmatrix} = \begin{bmatrix} 23 & 7.7 & 37.5 & 33.5 \end{bmatrix}$$

**Dimensiones:** $W^T_{1 \times 3} \cdot X_{3 \times 4} = Y_{1 \times 4}$

---

## Paralelismo con Machine Learning

El procesamiento en batch que hemos visto con las múltiples carreras de taxi es **exactamente** cómo funcionan las redes neuronales en la práctica:

### Ejemplo: Clasificación de imágenes en batch

In [ ]:
# Batch de 32 imágenes (cada imagen aplanada tiene 784 píxeles)
X_batch = np.random.rand(785, 32)  # 784 características + 1 para bias, 32 muestras
W = np.random.rand(1, 785)         # Una neurona con 785 pesos

Y_batch = W @ X_batch              # shape: (1, 32)
# Obtenemos 32 predicciones simultáneamente, una por cada imagen

**Ventajas del procesamiento en batch:**
- **Eficiencia computacional:** Procesar 100 muestras a la vez es mucho más rápido que procesarlas una por una
- **Estabilidad en el entrenamiento:** Los gradientes calculados sobre un batch son más estables que los de una sola muestra
- **Aprovechamiento del hardware:** GPUs están optimizadas para operaciones matriciales grandes

En el entrenamiento de redes neuronales, los tamaños de batch típicos son 32, 64, 128 o 256 muestras. El concepto es idéntico a nuestro ejemplo de las 4 carreras de taxi, solo que a mayor escala.

---

## Fórmula general

Para procesar $m$ características necesitamos $m$ pesos más un peso adicional para el sesgo:

$$y = w_0 + w_1 x_1 + w_2 x_2 + \cdots + w_m x_m$$

Para expresar esto en forma matricial, añadimos un 1 al vector de características:

$$y = w_0 \cdot 1 + w_1 \cdot x_1 + w_2 \cdot x_2 + \cdots + w_m \cdot x_m$$

En notación matricial:

$$y = W^T X = \begin{bmatrix} w_0 & w_1 & w_2 & \cdots & w_m \end{bmatrix} \begin{bmatrix} 1 \\ x_1 \\ x_2 \\ \vdots \\ x_m \end{bmatrix}$$

### Dimensiones para una sola muestra:

$$W^T_{1 \times (m+1)} \cdot X_{(m+1) \times 1} = y_{1 \times 1}$$

El resultado es un escalar (un número).

### Dimensiones para un batch de $n$ muestras:

$$W^T_{1 \times (m+1)} \cdot X_{(m+1) \times n} = Y_{1 \times n}$$

El resultado es un vector con $n$ predicciones, una para cada muestra del batch.

---

## Resumen conceptual

1. **Un perceptrón** es una neurona artificial que procesa entradas mediante una suma ponderada
2. **Una muestra:** El perceptrón toma $m$ características y produce una salida
3. **Batch de muestras:** Podemos procesar $n$ muestras simultáneamente usando la misma operación matricial
4. **El sesgo (bias)** se incorpora añadiendo un 1 a las entradas, lo que simplifica la notación
5. **En ML real:** Siempre se procesan batches de datos, no muestras individuales, por eficiencia y estabilidad

El ejemplo del taxista ilustra perfectamente estos conceptos: desde calcular el importe de una carrera (una muestra) hasta calcular los importes de múltiples carreras simultáneamente (batch processing), que es exactamente cómo funcionan las redes neuronales modernas.